# Delayed Monetary Losses: A Comparison of Two Procedures

This notebook replicates the analyses from the publication:

> Wan, H., Green, L., & Myerson, J. (2024). Delayed monetary losses: Do different procedures and measures assess the same construct?. *Behavioural Processes*, 105101. https://doi.org/10.1016/j.beproc.2024.105101

The original analyses were conducted in R and have been translated into a Python workflow using `pandas`, `numpy`, `statsmodels`, `lmfit`, and `seaborn`. The goal is to determine whether the Adjusting-Amount and the Delayed Losses Questionnaire (DLQ) procedures assess the same underlying construct.

In [ ]:
# Install necessary packages
import sys
!{sys.executable} -m pip install pandas numpy scipy lmfit statsmodels seaborn scikit-learn openpyxl

In [45]:
# --- Setup: Imports and Custom Functions ---
import pandas as pd
import numpy as np
import warnings
from lmfit import Model
from sklearn.metrics import auc as auc_calc
from scipy.stats import gmean
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format

# --- Custom Functions ---
def set_mattheme(ax):
    """Applies a consistent plot theme similar to the R version."""
    ax.set_title(ax.get_title(), fontsize=16, fontweight='bold')
    ax.set_xlabel(ax.get_xlabel(), fontsize=14, fontweight='bold')
    ax.set_ylabel(ax.get_ylabel(), fontsize=14, fontweight='bold')
    ax.tick_params(axis='both', which='major', labelsize=10)
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1)
    ax.set_facecolor('white')
    sns.despine(ax=ax)

def fit_nls_k(df):
    """Fits a simple hyperbola to individual Adj-Amt data to get log k."""
    def model_func(iv, k):
        return 1 / (1 + np.exp(k) * iv)
    
    model = Model(model_func)
    try:
        result = model.fit(df['value'], iv=df['iv'], k=-4)
        return result.params['k'].value
    except:
        return np.nan

# --- Data Loading and Processing ---

# Load and filter raw data
disc_raw = pd.read_csv("R Code/Data/DelayLoss.csv", index_col=0)
disc_df = disc_raw[
    (disc_raw['check'] == 7) & (disc_raw['provider'] != "MTurk")
].copy()

# The original 'id' column has the participant identifiers.
# We group by this column and use .ngroup() to create a new, sequential ID from 1 to N,
# which matches the logic from the original R script.
disc_df['id'] = disc_df.groupby('id').ngroup() + 1

# Drop the now-unnecessary 'provider' column
disc_df = disc_df.drop(columns=['provider'])

# --- Process Adj-Amt Data ---
adj_amt_processed = disc_df[disc_df['procedure'] == "aa"].groupby(['id', 'procedure', 'amt']).apply(
    lambda g: pd.Series({
        'atheoretical': auc_calc(g['iv'] / 108, g['value']),
        'k': fit_nls_k(g)
    })
).reset_index()

# --- Process MCQ Data ---
def score_mcq_logk(df):
    """
    Calculates theoretical log k for the Delayed Losses Questionnaire.
    The logic is now consistent with the delayed gains scoring method.
    """
    choices = df['value'].values
    k_values = df['iv'].values
    
    # Define questionnaire k-value boundaries for edge cases
    min_k = 9e-6  # Smallest k in the questionnaire design
    max_k = 0.014 # Largest k in the questionnaire design
    
    # Handle edge cases
    # Always chooses immediate loss (value=1) -> patient for losses -> smallest k
    if all(choices == 1): 
        return np.log(min_k)
    # Always chooses delayed loss (value=0) -> impulsive for losses -> largest k
    if all(choices == 0): 
        return np.log(max_k)
    
    # For participants who switch, find k that maximizes response consistency
    n_consistent = [sum((choices == 0) & (k_values <= k) | (choices == 1) & (k_values >= k)) for k in k_values]
    max_consistency = np.max(n_consistent)
    indifference_ks = k_values[np.where(n_consistent == max_consistency)]
    
    # Return log of the geometric mean, consistent with your preferred calculation
    return np.log(np.exp(np.mean(np.log(indifference_ks))))

mcq_processed = disc_df[disc_df['procedure'] == "mcq"].groupby(['id', 'amt']).apply(
    lambda g: pd.Series({
        'atheoretical': g['value'].sum(),
        'k': score_mcq_logk(g)
    })
).reset_index()
mcq_processed['procedure'] = 'mcq'

# --- Combine into Final Analysis Dataframe ---
behav = pd.concat([adj_amt_processed, mcq_processed]).sort_values(['id', 'procedure', 'amt']).reset_index(drop=True)

# --- Create Choice Pattern Dataframes ---
def get_grp(df):
    corr = df['value'].corr(np.log(df['iv']))
    mean_val = df['value'].mean()
    if pd.isna(corr): corr = 0
    
    if df['procedure'].iloc[0] == 'aa':
        if corr < 0 or mean_val == df['value'].min(): return "Typical"
        if corr > 0: return "Atypical"
        if mean_val == df['value'].max(): return "Always immediate"
    else: # mcq
        if corr > 0 or mean_val == 0: return "Typical"
        if corr < 0: return "Atypical"
        if mean_val == 1: return "Always immediate"
    return "Typical"

chc_pat_raw = disc_df[((disc_df['procedure'] == 'aa') & (disc_df['amt'] != 3)) |
                      ((disc_df['procedure'] == 'mcq') & (disc_df['amt'] != 2))
                     ].groupby(['procedure', 'id', 'amt']).apply(get_grp).reset_index(name='grp')

chc_pat = chc_pat_raw.groupby(['procedure', 'id'])['grp'].apply(
    lambda g: g.iloc[0] if g.nunique() == 1 else "Undefined"
).reset_index()

co = chc_pat.pivot(index='id', columns='procedure', values='grp')

# --- Group Level Data Frame ---
disc_grp_df = disc_df.copy()
disc_grp_df.loc[disc_grp_df['procedure'] == 'mcq', 'iv'] = np.log(disc_grp_df.loc[disc_grp_df['procedure'] == 'mcq', 'iv'])
disc_grp_df = disc_grp_df.groupby(['procedure', 'amt', 'iv']).agg(
    mean_sv=('value', 'mean'), med_sv=('value', 'median')
).reset_index().sort_values('procedure')

## Group-Level Analyses

This section visualizes the group-level discounting functions for both the Adjusting-Amount and DLQ procedures and reports the goodness-of-fit for the corresponding nonlinear models.

In [39]:
# --- Goodness-of-Fit (R-squared) ---
def hyperboloid_loss_fit(df):
    def model_func(iv, k, b): return 1 / (1 + np.exp(k) * iv)**b
    model = Model(model_func)
    return model.fit(df['med_sv'], iv=df['iv'], k=-4, b=1).rsquared

def logistic_fit(df):
    def model_func(iv, x, r): return 1 / (1 + np.exp(-(df['iv'] - x) * r))
    model = Model(model_func)
    return model.fit(df['mean_sv'], iv=df['iv'], x=-10, r=1).rsquared

print("--- Adj-Amt: R-squared ---")
r2_aa = disc_grp_df[disc_grp_df['procedure'] == 'aa'].groupby('amt').apply(hyperboloid_loss_fit)
print(r2_aa)

print("\n--- DLQ: R-squared ---")
r2_mcq = disc_grp_df[disc_grp_df['procedure'] == 'mcq'].groupby('amt').apply(logistic_fit)
print(r2_mcq)

# --- Amount Effect Test ---
print("--- Adj-Amt: Amount Effect (Large vs. Small) ---")
aa_data = behav[behav['procedure'] == 'aa'].copy()
aa_data['amt'] = aa_data['amt'].astype('category')
# GLM with Binomial family can handle proportional data
aa_data['atheoretical_adj'] = np.clip(aa_data['atheoretical'], 1e-5, 1 - 1e-5)
glm_beta_model = smf.glm("atheoretical_adj ~ -1 + C(amt)", data=aa_data, 
                         family=sm.families.Binomial()).fit()
print(glm_beta_model.t_test("C(amt)[3] - C(amt)[1] = 0"))


print("\n--- DLQ: Amount Effect (Large vs. Small) ---")
mcq_data = behav[behav['procedure'] == 'mcq'].copy()
mcq_data['amt'] = mcq_data['amt'].astype('category')
# Create proportion and provide total trials as weights
mcq_data['prop_immediate'] = mcq_data['atheoretical'] / 9.0
mcq_data['n_trials'] = 9
glm_binomial_model = smf.glm("prop_immediate ~ -1 + C(amt)", data=mcq_data, 
                             family=sm.families.Binomial(),
                             weights=mcq_data['n_trials']).fit()
print(glm_binomial_model.t_test("C(amt)[3] - C(amt)[1] = 0"))

--- Adj-Amt: R-squared ---
amt
1   0.953
2   0.953
3   0.970
dtype: float64

--- DLQ: R-squared ---
amt
1   0.984
2   0.984
3   0.976
dtype: float64
--- Adj-Amt: Amount Effect (Large vs. Small) ---
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.2510      0.146     -1.723      0.085      -0.537       0.035

--- DLQ: Amount Effect (Large vs. Small) ---
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0856      0.142     -0.601      0.548      -0.365       0.194


## Individual-Level Discounting Measures

This section examines the reliability of the discounting measures (i.e., correlations between amounts *within* each procedure) and their construct validity (i.e., correlations *between* the two procedures).

In [42]:
# --- Reliability Analysis ---
print("--- Reliability: Correlations Between Amounts ---")
behav_long = behav.melt(id_vars=['id', 'procedure', 'amt'], value_vars=['atheoretical', 'k'],
                        var_name='measure', value_name='score')
reliability_pivot = behav_long.pivot_table(index=['id', 'procedure', 'measure'], columns='amt', values='score')
print(reliability_pivot.groupby(['procedure', 'measure']).corr())


print("\n--- Reliability: Correlations Between Measures (Atheoretical vs. Theoretical) ---")
print(behav.groupby(['procedure', 'amt'])[['atheoretical', 'k']].corr().unstack().iloc[:, 1])

--- Reliability: Correlations Between Amounts ---
amt                            1     2     3
procedure measure      amt                  
aa        atheoretical 1   1.000 0.782 0.636
                       2   0.782 1.000 0.763
                       3   0.636 0.763 1.000
          k            1   1.000 0.797 0.627
                       2   0.797 1.000 0.763
                       3   0.627 0.763 1.000
mcq       atheoretical 1   1.000 0.866 0.806
                       2   0.866 1.000 0.893
                       3   0.806 0.893 1.000
          k            1   1.000 0.836 0.747
                       2   0.836 1.000 0.840
                       3   0.747 0.840 1.000

--- Reliability: Correlations Between Measures (Atheoretical vs. Theoretical) ---
procedure  amt
aa         1     -0.960
           2     -0.968
           3     -0.967
mcq        1     -0.981
           2     -0.982
           3     -0.977
Name: (atheoretical, k), dtype: float64


## Analysis of Choice Patterns

This section categorizes participants based on their response patterns (e.g., typical, always immediate, atypical) and examines the consistency of these patterns across the two procedures.

In [46]:
# --- Choice Pattern Analysis ---
print("--- Frequencies of Choice Patterns by Procedure ---")
print(chc_pat.groupby('procedure')['grp'].value_counts(normalize=True))

print("\n--- Contingency Table of Choice Patterns (Adj-Amt vs. DLQ) ---")
crosstab = pd.crosstab(co['aa'], co['mcq'])
print(crosstab)

print("\n--- Cross-Tabulation with Expected Frequencies ---")
chi2, p, dof, expected = chi2_contingency(crosstab)
print("Expected Frequencies:")
print(pd.DataFrame(expected, index=crosstab.index, columns=crosstab.columns))

--- Frequencies of Choice Patterns by Procedure ---
procedure  grp             
aa         Typical            0.877
           Undefined          0.104
           Atypical           0.019
mcq        Typical            0.759
           Always immediate   0.155
           Undefined          0.084
           Atypical           0.002
Name: proportion, dtype: float64

--- Contingency Table of Choice Patterns (Adj-Amt vs. DLQ) ---
mcq        Always immediate  Atypical  Typical  Undefined
aa                                                       
Atypical                  2         0        5          1
Typical                  59         1      288         30
Undefined                 6         0       34          5

--- Cross-Tabulation with Expected Frequencies ---
Expected Frequencies:
mcq        Always immediate  Atypical  Typical  Undefined
aa                                                       
Atypical              1.244     0.019    6.070      0.668
Typical              58.761     0